In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [9]:
import pandas as pd
import numpy as np
import pickle
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split 
from sklearn.metrics import classification_report

def save_obj(obj, name ):
    with open(name + '.pkl', 'wb') as f:
        pickle.dump(obj, f, pickle.HIGHEST_PROTOCOL)

def load_obj(name ):
    try:
        with open(name + '.pkl', 'rb') as f:
            return pickle.load(f)    
    except FileNotFoundError as e:
        return False;

In [3]:
data = pd.read_csv('/content/drive/MyDrive/SIAP/data/prepared/final_vectors_with_buckets.csv')
data = data.dropna(subset=['REASONDESCRIPTION'])

reasons = data['REASONDESCRIPTION'].unique()
onehot_encoder = OneHotEncoder()
onehot_encoder.fit(reasons.reshape(len(reasons), 1))


OneHotEncoder(categories='auto', drop=None, dtype=<class 'numpy.float64'>,
              handle_unknown='error', sparse=True)

In [4]:
X = data[data.columns.difference(['PATIENT', 'IS_SELECTED_DEATH_REASON', 'REASONDESCRIPTION'])].to_numpy()
y = onehot_encoder.transform(data['REASONDESCRIPTION'].to_numpy().reshape(-1, 1)).toarray()


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state = 42)

.

.

**NN MULTILABEL CLASSIFICATOR**

.

.


.


In [14]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, BatchNormalization
from keras.optimizers import Adam
from keras.utils import to_categorical
import tensorflow as tf
from keras.callbacks import ModelCheckpoint 
from datetime import datetime 
from keras.metrics import BinaryAccuracy, Precision, Recall, AUC

num_epochs = 20
num_batch_size = 64

METRICS = [
      BinaryAccuracy(name='accuracy'),
      Precision(name='precision'),
      Recall(name='recall'),
      AUC(name='auc'),
      AUC(name='prc', curve='PR'), # precision-recall curve
]

# Notes:
# BatchNormalization recall from 0.5 to 0.8
# tf.keras.losses.CategoricalCrossentropy(from_logits=True) 0.95 recall, loss 0 at start...

def build_multilabel_model(input_dimension, output_dimensions):
    model = Sequential()
    model.add(Dense(2046, input_shape=(input_dimension,)))
    model.add(BatchNormalization())
    model.add(Activation('selu'))
    model.add(Dropout(0.2))
    model.add(Dense(516))
    model.add(BatchNormalization())
    model.add(Activation('selu'))
    model.add(Dropout(0.2))
    model.add(Dense(128))
    model.add(BatchNormalization())
    model.add(Activation('selu'))
    model.add(Dropout(0.2))
    # model.add(Dense(output_dimensions))
    # model.add(Activation('sigmoid'))
    model.add(Dense(output_dimensions, activation='softmax'))
    # Compile the model
    # model.compile(loss=tf.keras.losses.BinaryCrossentropy() , metrics=['accuracy'], optimizer='adam')

    model.compile(loss='categorical_crossentropy' ,metrics=METRICS, optimizer='adam') 
    return model

nn_model = build_multilabel_model(X_train.shape[1], y.shape[1])
# nn_model.summary()

In [15]:
nn_model.fit(X_train, y_train, batch_size=num_batch_size, epochs=num_epochs, validation_split=0.2, verbose=1)

Epoch 1/20
192/192 [==============================] - 15s 63ms/step - loss: 1.2523 - accuracy: 0.9818 - precision: 0.7990 - recall: 0.6246 - auc: 0.9423 - prc: 0.7455 - val_loss: 0.9331 - val_accuracy: 0.9819 - val_precision: 0.8973 - val_recall: 0.5381 - val_auc: 0.9852 - val_prc: 0.8414
Epoch 2/20
192/192 [==============================] - 11s 60ms/step - loss: 0.7129 - accuracy: 0.9863 - precision: 0.8348 - recall: 0.7525 - auc: 0.9844 - prc: 0.8651 - val_loss: 0.6773 - val_accuracy: 0.9868 - val_precision: 0.8367 - val_recall: 0.7683 - val_auc: 0.9874 - val_prc: 0.8734
Epoch 3/20
192/192 [==============================] - 11s 59ms/step - loss: 0.6269 - accuracy: 0.9875 - precision: 0.8488 - recall: 0.7766 - auc: 0.9877 - prc: 0.8849 - val_loss: 0.6777 - val_accuracy: 0.9865 - val_precision: 0.8240 - val_recall: 0.7738 - val_auc: 0.9864 - val_prc: 0.8720
Epoch 4/20
192/192 [==============================] - 11s 59ms/step - loss: 0.5855 - accuracy: 0.9877 - precision: 0.8480 - recall

In [16]:
y_pred = nn_model.predict(X_test, batch_size=64, verbose=1)
y_pred = (y_pred > 0.5)

print(classification_report(y_test, y_pred, zero_division=0, target_names=[reason.split("x0_")[1] for reason in onehot_encoder.get_feature_names()]))


60/60 [==============================] - 1s 17ms/step
                                                          precision    recall  f1-score   support

                          Alzheimer's disease (disorder)       0.29      0.33      0.31         6
                    Burn injury(morphologic abnormality)       0.18      0.06      0.09        33
                                                COVID-19       0.68      0.60      0.64       402
                                          Cardiac Arrest       0.18      0.06      0.09        86
             Chronic congestive heart failure (disorder)       0.95      0.97      0.96       175
               Chronic obstructive bronchitis (disorder)       0.85      0.73      0.79        56
                              Concussion injury of brain       0.14      0.02      0.04        95
                  Death due to acute respiratory failure       1.00      0.83      0.91         6
                                     Death due to sepsis       

.

.

**XGBOOST MULTILABEL CLASSIFICATOR**

.

.


.


In [17]:
## XGBoost

xgb_y_train =  [np.where(arr == 1)[0][0] for arr in y_train]
xgb_y_test =  [np.where(arr == 1)[0][0] for arr in y_test]


import xgboost as xgb
dtrain = xgb.DMatrix(X_train, xgb_y_train)
dtest = xgb.DMatrix(X_test, xgb_y_test)

params = {
    'max_depth':10,
    # 'min_child_weight': 1,
    # 'eta':.3,
    # 'subsample': 0.6,
    # 'colsample_bytree': 0.6,
    'objective':'multi:softmax',
    # 'objective':'multi:softprob',
    'num_class': y.shape[1],
    'eval_metric' : 'merror'
}

model = xgb.train(
    params,
    dtrain,
    #evals=[(dtest, "Test")], ovde treba koristiti validacioni skup
    evals=[(dtrain, "Train")],
    early_stopping_rounds=35,
    num_boost_round=999
)

[0]	Train-merror:0.152803
Will train until Train-merror hasn't improved in 35 rounds.
[1]	Train-merror:0.130248
[2]	Train-merror:0.11558
[3]	Train-merror:0.103585
[4]	Train-merror:0.091851
[5]	Train-merror:0.082725
[6]	Train-merror:0.074707
[7]	Train-merror:0.068057
[8]	Train-merror:0.058931
[9]	Train-merror:0.052151
[10]	Train-merror:0.042177
[11]	Train-merror:0.035984
[12]	Train-merror:0.02914
[13]	Train-merror:0.021773
[14]	Train-merror:0.017666
[15]	Train-merror:0.013885
[16]	Train-merror:0.011734
[17]	Train-merror:0.008735
[18]	Train-merror:0.006063
[19]	Train-merror:0.00515
[20]	Train-merror:0.004237
[21]	Train-merror:0.003259
[22]	Train-merror:0.002738
[23]	Train-merror:0.002216
[24]	Train-merror:0.00176
[25]	Train-merror:0.001499
[26]	Train-merror:0.001304
[27]	Train-merror:0.001239
[28]	Train-merror:0.001239
[29]	Train-merror:0.001173
[30]	Train-merror:0.001108
[31]	Train-merror:0.001043
[32]	Train-merror:0.001043
[33]	Train-merror:0.001043
[34]	Train-merror:0.001043
[35]	Trai

In [18]:
results = model.predict(dtest)
print(classification_report(xgb_y_test, results, zero_division=0, target_names=[reason.split("x0_")[1] for ind, reason in enumerate(onehot_encoder.get_feature_names()) if ind not in [8, 9]  ]))

                                                          precision    recall  f1-score   support

                          Alzheimer's disease (disorder)       0.50      0.50      0.50         6
                    Burn injury(morphologic abnormality)       0.00      0.00      0.00        33
                                                COVID-19       0.82      0.85      0.83       402
                                          Cardiac Arrest       0.38      0.06      0.10        86
             Chronic congestive heart failure (disorder)       0.94      1.00      0.97       175
               Chronic obstructive bronchitis (disorder)       0.88      0.80      0.84        56
                              Concussion injury of brain       0.31      0.05      0.09        95
                  Death due to acute respiratory failure       1.00      0.83      0.91         6
  Familial Alzheimer's disease of early onset (disorder)       0.56      0.56      0.56         9
Fracture of the ver